In [1]:
from econml.dml import RLearner 


ImportError: cannot import name 'RLearner' from 'econml.dml' (/Users/jason/miniconda3/envs/pytorch/lib/python3.10/site-packages/econml/dml/__init__.py)

In [ ]:
I saw a example : """import numpy as np from sklearn.linear_model import LinearRegression from econml.dml._rlearner import RLearner from econml.sklearnextensions.model_selection import SingleModelSelector from sklearn.base import clone class ModelFirst:     def init(self, model):         self._model = clone(model, safe=False)     def fit(self, X, W, Y, sample_weight=None):         self._model.fit(np.hstack([X, W]), Y)         return self     def predict(self, X, W):         return self._model.predict(np.hstack([X, W])) class ModelSelector(SingleModelSelector):     def init(self, model):         self._model = ModelFirst(model)     def train(self, is_selecting, folds, X, W, Y, sample_weight=None):         self._model.fit(X, W, Y, sample_weight=sample_weight)         return self     @property     def best_model(self):         return self._model     @property     def best_score(self):         return 0 class ModelFinal:     def fit(self, X, T, T_res, Y_res, sample_weight=None, freq_weight=None, sample_var=None):         self.model = LinearRegression(fit_intercept=False).fit(X * T_res.reshape(-1, 1),                                                                Y_res)         return self     def predict(self, X):         return self.model.predict(X) class RLearner(_RLearner):     def genmodel_y(self):         return ModelSelector(LinearRegression())     def genmodel_t(self):         return ModelSelector(LinearRegression())     def genrlearner_model_final(self):         return ModelFinal() np.random.seed(123) X = np.random.normal(size=(1000, 3)) y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.01, size=(1000,)) est = RLearner(cv=2, discrete_outcome=False, discrete_treatment=False,                treatment_featurizer=None, categories='auto', random_state=None) est.fit(y, X[:, 0], X=np.ones((X.shape[0], 1)), W=X[:, 1:])""" from this webpage: https://econml.azurewebsites.net/_autosummary/econml.dml._rlearner.html#econml.dml._rlearner._RLearner. 

I think it could have similar implementation like other XLearner, TLearner to use the regressor and classifier from the _get_regressor()  and getclassifier() methods as basemodel.  

Could you make the RLearner available?

In [2]:
import torch
import numpy as np

# Helper function (provided in your earlier context)
def numpy_adj_to_torch_sparse_tensor(adj_matrix):
    """
    checked!
    For data in .mat
    """
    rows, cols = np.nonzero(adj_matrix)
    indices = np.stack((rows, cols), axis=0)
    indices = torch.from_numpy(indices.astype(np.int64))
    num_edges = indices.shape[1]
    values = torch.ones(num_edges, dtype=torch.float32) # Assuming all 1s for this example
    
    shape = torch.Size(adj_matrix.shape)
    sparse_tensor = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float32)
    return sparse_tensor

# --- Test Data Setup ---

# 1. Define an original (dense) adjacency matrix
# Let's say we have 6 nodes.
# Adjacency matrix:
#    0 1 2 3 4 5
# 0: 0 1 0 0 1 0
# 1: 1 0 1 0 0 0
# 2: 0 1 0 1 0 0
# 3: 0 0 1 0 0 1
# 4: 1 0 0 0 0 0
# 5: 0 0 0 1 0 0
# Example: Edges are (0,1), (0,4), (1,0), (1,2), (2,1), (2,3), (3,2), (3,5), (4,0), (5,3)
original_adj_np = np.array([
    [0, 1, 0, 0, 1, 0],
    [1, 0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1],
    [1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0]
], dtype=np.float32)

# Convert to sparse tensor (this will be A_original)
A_original = numpy_adj_to_torch_sparse_tensor(original_adj_np)
print(f"Original A (sparse): {A_original}")
print(f"Original A indices:\n{A_original._indices()}")
print(f"Original A values:\n{A_original._values()}\n")

# 2. Simulate which rows are "non-null"
# Let's say rows 1 and 4 are considered "null" and we want to remove them.
# So, nodes 0, 2, 3, 5 are non-null.
# Original indices: 0, 1, 2, 3, 4, 5
# Non-null indicator: T, F, T, T, F, T
non_null_indicator = torch.tensor([True, False, True, True, False, True], dtype=torch.bool)
print(f"Non-null indicator: {non_null_indicator}\n")

# --- Your code snippet to test ---
print("--- Applying the code snippet ---")

non_null_indices = torch.nonzero(non_null_indicator).squeeze(1)
print(f"non_null_indices (original indices of non-null rows): {non_null_indices}\n")

A_indices = A_original._indices()
A_values = A_original._values()

mask_rows = torch.isin(A_indices[0], non_null_indices)
mask_cols = torch.isin(A_indices[1], non_null_indices)
print(f"mask_rows (source node in non-null set): {mask_rows}")
print(f"mask_cols (target node in non-null set): {mask_cols}\n")

valid_edges_mask = mask_rows & mask_cols
print(f"valid_edges_mask (both source and target in non-null set): {valid_edges_mask}\n")

filtered_A_indices = A_indices[:, valid_edges_mask]
filtered_A_values = A_values[valid_edges_mask]
print(f"filtered_A_indices (original indices of kept edges):\n{filtered_A_indices}")
print(f"filtered_A_values (values of kept edges):\n{filtered_A_values}\n")

original_to_new_map = -torch.ones(A_original.shape[0], dtype=torch.long)
original_to_new_map[non_null_indices] = torch.arange(len(non_null_indices))
print(f"original_to_new_map (maps original index to new index):\n{original_to_new_map}\n")

remap_rows = original_to_new_map[filtered_A_indices[0]]
remap_cols = original_to_new_map[filtered_A_indices[1]]
print(f"remap_rows (new indices for source nodes):\n{remap_rows}")
print(f"remap_cols (new indices for target nodes):\n{remap_cols}\n")

new_shape = torch.Size((len(non_null_indices), len(non_null_indices)))
print(f"new_shape: {new_shape}\n")

A_filtered = torch.sparse_coo_tensor(torch.stack((remap_rows, remap_cols), dim=0), filtered_A_values, new_shape, dtype=torch.float32)

print("--- Result ---")
print(f"Filtered A (sparse): {A_filtered}")
print(f"Filtered A indices:\n{A_filtered._indices()}")
print(f"Filtered A values:\n{A_filtered._values()}")

# Verify by converting to dense and comparing
print("\n--- Verification ---")
print("Original A (dense):\n", A_original.to_dense().numpy())
print("\nExpected filtered A (dense, by manual selection):")
# Manual selection: original nodes 0, 2, 3, 5 become new nodes 0, 1, 2, 3
expected_dense_filtered_A = np.array([
    # Original nodes: 0, 2, 3, 5
    # New nodes:      0, 1, 2, 3
    [0, 0, 0, 0],  # 0 (was 0) connects to no other non-null
    [0, 0, 1, 0],  # 1 (was 2) connects to 2 (was 3)
    [0, 1, 0, 1],  # 2 (was 3) connects to 1 (was 2) and 3 (was 5)
    [0, 0, 1, 0]   # 3 (was 5) connects to 2 (was 3)
])
print(expected_dense_filtered_A)

print("\nResulting filtered A (dense):\n", A_filtered.to_dense().numpy())

# Assert for correctness
assert np.array_equal(A_filtered.to_dense().numpy(), expected_dense_filtered_A)
print("\nAssertion successful: Filtered A matches expected result!")

Original A (sparse): tensor(indices=tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 5],
                       [1, 4, 0, 2, 1, 3, 2, 5, 0, 3]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
       size=(6, 6), nnz=10, layout=torch.sparse_coo)
Original A indices:
tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 5],
        [1, 4, 0, 2, 1, 3, 2, 5, 0, 3]])
Original A values:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Non-null indicator: tensor([ True, False,  True,  True, False,  True])

--- Applying the code snippet ---
non_null_indices (original indices of non-null rows): tensor([0, 2, 3, 5])

mask_rows (source node in non-null set): tensor([ True,  True, False, False,  True,  True,  True,  True, False,  True])
mask_cols (target node in non-null set): tensor([False, False,  True,  True, False,  True,  True,  True,  True,  True])

valid_edges_mask (both source and target in non-null set): tensor([False, False, False, False, False,  True,  True,  True, False,  True])

filtered_A_indices

In [ ]:
def create_path(base_path, j):
    parts = base_path.split('mats/')
    new_path = parts[0] + 'mats/' + f'dataset{j}/' + parts[1]
    return new_path

# Example usage
original_path = 'datasets/mats/adf/adf/Syn/1.txt'
j = 5
new_path = create_path(original_path, j)
print(new_path)  # Output: datasets/mats/dataset5/Syn/1.txt

datasets/mats/dataset5/adf/adf/Syn/1.txt


In [3]:
import numpy as np

# Example 1: array of shape (1, 10)
a = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
print("Original shape a:", a.shape)

a_squeezed = np.squeeze(a)
print("Squeezed shape a:", a_squeezed.shape)
print("Squeezed a:", a_squeezed)

# Example 2: array of shape (10, 1)
b = np.array([[i] for i in range(10)])
print("\nOriginal shape b:", b.shape, len(b.shape))

b_squeezed = np.squeeze(b)
print("Squeezed shape b:", b_squeezed.shape, len(b_squeezed.shape))
print("Squeezed b:", b_squeezed)
print("squeezed squeezed b:", np.squeeze(b_squeezed))


Original shape a: (1, 10)
Squeezed shape a: (10,)
Squeezed a: [ 1  2  3  4  5  6  7  8  9 10]

Original shape b: (10, 1) 2
Squeezed shape b: (10,) 1
Squeezed b: [0 1 2 3 4 5 6 7 8 9]
squeezed squeezed b: [0 1 2 3 4 5 6 7 8 9]


In [1]:
import scipy.io as sio

dataset_mat_fn = '/home/jason/coding/NetDeconf_main_hao/datasets/mats/test_imputed=0/Syn/mice/p=0.0_k=2_seed=70.pt.mat'
# dataset_mat_fn = '/home/jason/coding/NetDeconf_main_hao/datasets/mats/test_imputed=0/Syn/mice/p=0.0_k=2_seed=70.pt.mat'
# dataset_mat_fn = '/home/jason/coding/NetDeconf_main_hao/datasets/mats/test_imputed=0/Syn/mice/p=0.1_k=0_seed=919.pt.mat'

data = sio.loadmat(dataset_mat_fn)

In [1]:
import numpy as np

# Example boolean index array
bool_indices = np.array([True, False, True, True, False, False, True])

print("Original boolean array:", bool_indices)
print("Data type:", bool_indices.dtype)

int_indices_where = np.where(bool_indices)[0]

print("\nUsing np.where():")
print("Integer indices:", int_indices_where)
print("Data type:", int_indices_where.dtype)

Original boolean array: [ True False  True  True False False  True]
Data type: bool

Using np.where():
Integer indices: [0 2 3 6]
Data type: int64


In [4]:
import os

########################################## Method Settting  ################################
results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

method = 'NetDeconf'
dataset_dir = 'Flickr'
missing_p = '0.3'
############################################################################################

balu_dir = '/mnt/vast-kisski/projects/kisski-tib-activecl/BaLu'
source_dir = 'datasets/exps/'
target_dir = 'datasets/mats/'
os.makedirs(target_dir, exist_ok=True)

if os.path.exists(balu_dir):
    source_dir = os.path.join(balu_dir, source_dir)
    target_dir = os.path.join(balu_dir, target_dir)

src_dataset = os.path.join(source_dir, dataset_dir)   # source dataset dir
tar_dataset = os.path.join(target_dir, dataset_dir)   # target dataset dir

dataset_result_dir = os.path.join(results_dir, dataset_dir)
os.makedirs(dataset_result_dir, exist_ok=True)

method_result_dir = os.path.join(dataset_result_dir, method)
os.makedirs(method_result_dir, exist_ok=True)

for fn in os.listdir(src_dataset):
    if f"p={missing_p}" not in fn:
        continue
    for impute in ['no', 'mean', 'knn', 'mice', 'missforest', 'gain']:
        method_impute_dir = os.path.join(method_result_dir, impute)
        os.makedirs(method_impute_dir, exist_ok=True)
        
        tar_dataset_impute = os.path.join(tar_dataset, impute)
        data_mat_fn = os.path.join(tar_dataset_impute, fn+".mat")
        parts = data_mat_fn.split("mats/")[1].split("/")
        # dataset = parts[0]; method = parts[1]
        identifier = parts[2].split(".pt")[0]
        one_result_fn = os.path.join(method_impute_dir, identifier)
        print(data_mat_fn)
        print(one_result_fn)
            

datasets/mats/Flickr/no/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/no/p=0.3_k=9_seed=300
datasets/mats/Flickr/mean/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/mean/p=0.3_k=9_seed=300
datasets/mats/Flickr/knn/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/knn/p=0.3_k=9_seed=300
datasets/mats/Flickr/mice/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/mice/p=0.3_k=9_seed=300
datasets/mats/Flickr/missforest/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/missforest/p=0.3_k=9_seed=300
datasets/mats/Flickr/gain/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/gain/p=0.3_k=9_seed=300
datasets/mats/Flickr/no/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/no/p=0.3_k=6_seed=36
datasets/mats/Flickr/mean/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/mean/p=0.3_k=6_seed=36
datasets/mats/Flickr/knn/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/knn/p=0.3_k=6_seed=36
datasets/mats/Flickr/mice/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/mice/p=0.3_k=6_seed=3

In [2]:
import os

########################################## Method Settting  ################################
results_dir = 'results'
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
method = 'NetDeconf'
dataset_dir = 'Flickr'
missing_p = '0.3'
############################################################################################

balu_dir = '/mnt/vast-kisski/projects/kisski-tib-activecl/BaLu'
source_dir = 'datasets/exps/'
target_dir = 'datasets/mats/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

if os.path.exists(balu_dir):
    source_dir = os.path.join(balu_dir, source_dir)
    target_dir = os.path.join(balu_dir, target_dir)

src_dataset = os.path.join(source_dir, dataset_dir)   # source dataset dir
tar_dataset = os.path.join(target_dir, dataset_dir)   # target dataset dir

dataset_result_dir = os.path.join(results_dir, dataset_dir)
if not os.path.exists(dataset_result_dir):
    os.mkdir(dataset_result_dir)

method_result_dir = os.path.join(dataset_result_dir, method)
if not os.path.exists(method_result_dir):
    os.mkdir(method_result_dir)

for fn in os.listdir(src_dataset):
    if f"p={missing_p}" not in fn:
        continue
    for impute in ['no', 'mean', 'knn', 'mice', 'missforest', 'gain']:
        method_impute_dir = os.path.join(method_result_dir, impute)
        if not os.path.exists(method_impute_dir):
            os.mkdir(method_impute_dir)
        
        tar_dataset_impute = os.path.join(tar_dataset, impute)
        data_mat_fn = os.path.join(tar_dataset_impute, fn+".mat")
        parts = data_mat_fn.split("mats/")[1].split("/")
        # dataset = parts[0]; method = parts[1]
        identifier = parts[2].split(".pt")[0]
        one_result_fn = os.path.join(method_impute_dir, identifier)
        print(data_mat_fn)
        print(one_result_fn)
            

datasets/mats/Flickr/no/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/no/p=0.3_k=9_seed=300
datasets/mats/Flickr/mean/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/mean/p=0.3_k=9_seed=300
datasets/mats/Flickr/knn/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/knn/p=0.3_k=9_seed=300
datasets/mats/Flickr/mice/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/mice/p=0.3_k=9_seed=300
datasets/mats/Flickr/missforest/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/missforest/p=0.3_k=9_seed=300
datasets/mats/Flickr/gain/p=0.3_k=9_seed=300.pt.mat
results/Flickr/NetDeconf/gain/p=0.3_k=9_seed=300
datasets/mats/Flickr/no/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/no/p=0.3_k=6_seed=36
datasets/mats/Flickr/mean/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/mean/p=0.3_k=6_seed=36
datasets/mats/Flickr/knn/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/knn/p=0.3_k=6_seed=36
datasets/mats/Flickr/mice/p=0.3_k=6_seed=36.pt.mat
results/Flickr/NetDeconf/mice/p=0.3_k=6_seed=3

AttributeError: 'dict' object has no attribute 'b'